# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [16]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key
import warnings
warnings.filterwarnings("ignore")

In [17]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,nar'yan-mar,67.6713,53.0870,6.97,88,30,2.92,RU,1693438424
1,1,shakawe,-18.3667,21.8500,13.16,31,0,2.57,BW,1693438424
2,2,matoury,4.8483,-52.3317,27.98,78,0,0.51,GF,1693438425
3,3,susuman,62.7833,148.1667,17.65,57,100,10.23,RU,1693438425
4,4,puerto ayora,-0.7393,-90.3518,24.75,91,71,4.47,EC,1693438426


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [20]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
coordinates = {
    "Lat": city_data_df["Lat"],
    "Lng": city_data_df["Lng"],
    "City": city_data_df["City"],
    "Humidity": city_data_df["Humidity"]
}
coordinates_df =pd.DataFrame(coordinates)
#coordinates_df.head()
cities_map = coordinates_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.01,
    color = "City"
)
# Display the map
# YOUR CODE HERE
cities_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [26]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
ideal_weather_condition = city_data_df.loc[((city_data_df["Max Temp"]<=35.0) & (city_data_df["Max Temp"]>25.0)), ["City","Lat","Lng","Max Temp","Humidity","Cloudiness","Wind Speed","Country","Date"]]
# Drop any rows with null values
# YOUR CODE HERE
ideal_weather_condition = ideal_weather_condition.dropna(how='any')
#ideal_weather_condition.count()
# Display sample data
# YOUR CODE HERE
ideal_weather_condition.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,matoury,4.8483,-52.3317,27.98,78,0,0.51,GF,1693438425
7,anna regina,7.2644,-58.5077,27.50,78,0,3.95,GY,1693438428
16,hawaiian paradise park,19.5933,-154.9731,27.80,88,100,6.17,US,1693438433
19,spring creek,40.7266,-115.5859,28.23,12,0,4.68,US,1693438396
22,la ronge,55.1001,-105.2842,29.14,37,40,4.12,CA,1693438437


### Step 3: Create a new DataFrame called `hotel_df`.

In [21]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = city_data_df[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df.insert(5, "Hotel Name","")
# Display sample data
# YOUR CODE HERE
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,nar'yan-mar,RU,67.6713,53.0870,88,
1,shakawe,BW,-18.3667,21.8500,31,
2,matoury,GF,4.8483,-52.3317,78,
3,susuman,RU,62.7833,148.1667,57,
4,puerto ayora,EC,-0.7393,-90.3518,91,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [22]:
# Set parameters to search for a hotel
radius = 10000# YOUR CODE HERE
limit = 3
params = {
    # YOUR CODE HERE
    "limit":limit,
    "apiKey":geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    latitude =row["Lat"]
    longitude = row["Lng"]
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}" # YOUR CODE HERE
    params["bias"] = f"proximity:{longitude},{latitude}" # YOUR CODE HERE
    params["categories"] = f"accommodation.hotel"
   
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params) # YOUR CODE HERE
    
    # Convert the API response to JSON format
    name_address = name_address.json()# YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
nar'yan-mar - nearest hotel: Гостиница "Агат"
shakawe - nearest hotel: Drotsky's Cabins
matoury - nearest hotel: La Marmotte
susuman - nearest hotel: No hotel found
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
waitangi - nearest hotel: Hotel Chathams
ola - nearest hotel: No hotel found
anna regina - nearest hotel: Jaigobin Hotel
adamstown - nearest hotel: No hotel found
papatowai - nearest hotel: No hotel found
talnakh - nearest hotel: Талнах
chonchi - nearest hotel: Hotel Huildín
broome - nearest hotel: No hotel found
bethel - nearest hotel: Hampton Inn Danbury
beloyarskiy - nearest hotel: Сибирь
hamilton - nearest hotel: North Vista Manor
hawaiian paradise park - nearest hotel: No hotel found
invercargill - nearest hotel: The Grand
port alfred - nearest hotel: No hotel found
spring creek - nearest hotel: No hotel found
port-aux-francais - nearest hotel: Keravel
margaret river - nearest hotel: Margaret River Hotel
la ronge - nearest hotel: L

,City,Country,Lat,Lng,Humidity,Hotel Name
0,nar'yan-mar,RU,67.6713,53.0870,88,"Гостиница ""Агат"""
1,shakawe,BW,-18.3667,21.8500,31,Drotsky's Cabins
2,matoury,GF,4.8483,-52.3317,78,La Marmotte
3,susuman,RU,62.7833,148.1667,57,No hotel found
4,puerto ayora,EC,-0.7393,-90.3518,91,Hostal La Mirada De Solitario George
...,...,...,...,...,...,...
555,kushiro,JP,42.9750,144.3747,100,ANA Hotel Kushiro
556,corner brook,CA,48.9667,-57.9484,98,No hotel found
557,colac,AU,-38.3333,143.5833,84,No hotel found
558,nantucket,US,41.2835,-70.0995,97,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [24]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

coordinates = {
    "Lat": hotel_df["Lat"],
    "Lng": hotel_df["Lng"],
    "Humidity": hotel_df["Humidity"],
    "City": hotel_df["City"],
    "Country": hotel_df["Country"],
    "HotelName": hotel_df["Hotel Name"]
 
}
coordinates_df =pd.DataFrame(coordinates)
coordinates_df.head()
cities_map = coordinates_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.01,
    color = "City"
)
# Display the map
# YOUR CODE HERE
cities_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)